<a href="https://colab.research.google.com/github/rizkairianty/eda-text-classification/blob/main/Copy_of_SMOTE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install scikeras

In [9]:
!pip install sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 5.6 MB/s eta 0:00:00


In [3]:
!pip install -U nltk

In [4]:
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

from keras.layers import Dense, Dropout
from keras.models import Model, Sequential

from keras.layers import Bidirectional
import keras.layers as layers
from keras.models import Sequential

from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.combine import SMOTETomek, SMOTEENN
from imblearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier

In [5]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, RandomizedSearchCV

In [34]:
from os import listdir
from os.path import isfile, join, isdir
import re
import pickle
import tensorflow as tf
import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier
from keras.callbacks import EarlyStopping
from sklearn.metrics import make_scorer

In [10]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors, Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec

from drive.MyDrive.aathesis.func import *
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

In [ ]:
nltk.download('stopwords')
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('indonesian'))
factory = StemmerFactory()
stemmer = factory.create_stemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
data_df = text_to_df('./drive/MyDrive/aathesis/increment_datasets_f2/pc/orig_full.txt')
data_df = data_df[:2500]

In [ ]:
def clean_text(text):
  text = text.lower()
  text = REPLACE_BY_SPACE_RE.sub(' ', text)
  text = BAD_SYMBOLS_RE.sub('', text)
  text = ' '.join(word for word in text.split() if word not in STOPWORDS)
  text = stemmer.stem(text)
  return text

In [ ]:
data_df['text'] = data_df['text'].apply(clean_text)

In [ ]:
data_df.to_csv("./drive/MyDrive/aathesis/data_clean.txt", sep='\t', index = None, header = None)

In [ ]:
# test_df = text_to_df('/content/drive/MyDrive/aathesis/increment_datasets_f2/pc/test.txt')
# test_df['text'] = test_df['text'].apply(clean_text)

In [11]:
data_df = text_to_df('/content/drive/MyDrive/aathesis/data_clean.txt')
test_df = text_to_df('/content/drive/MyDrive/aathesis/test_clean.txt')

In [12]:
data_df.drop(data_df[data_df['text'].isna() == True].index, inplace = True)
test_df.drop(test_df[test_df['text'].isna() == True].index, inplace = True)

## Load GloVe Embedding

In [13]:
word2vec_len = 300
sentence_length = 30

In [39]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(data_df['text'])

test_vectorizer = TfidfVectorizer()
X_test = vectorizer.transform(test_df['text'])

Y_train = data_df['label']
Y_test = test_df['label']

In [14]:
my_tokenizer = Tokenizer()
my_tokenizer.fit_on_texts(data_df['text'])
vocab_size = len(my_tokenizer.word_index)+1
encoded_vocab = my_tokenizer.texts_to_sequences(data_df['text'])
print(encoded_vocab)

[[152, 143, 553, 2020, 603, 115, 383, 6, 98, 316, 2021, 219, 316, 2953, 152, 23, 5, 266, 514, 52, 5, 21, 196, 50, 4, 45, 515, 2954, 1, 289, 347, 2955], [654, 871, 2022, 2956, 2957, 2958, 197, 2959, 300, 172, 2960], [30, 167, 13, 1315, 6, 1, 12, 696, 140, 56, 2961, 1, 3, 29, 2, 24, 995, 136, 2962, 1, 301], [2963, 655, 1, 2964, 237, 267, 1, 19, 1595, 107], [1596, 410, 1316, 108, 411, 472, 348, 2965, 2966, 473, 474, 656, 1, 473, 1317, 268, 410], [2, 116, 4, 2, 657, 2967, 290, 697, 14, 53, 658, 65, 84], [51, 411, 1139, 996, 302, 698], [88, 19, 168, 2, 3, 997, 57, 100, 63, 1140, 2023, 4, 101, 90, 1, 5, 46, 2, 103, 50, 5, 77, 2024, 2968, 2025], [28, 31, 137, 192, 92, 998, 778, 42, 13, 2026, 6, 999, 998, 778, 42, 13, 2026, 872, 516, 1597, 699, 291, 16, 8, 9, 8, 141, 74], [1141], [2969, 193, 2970, 1142, 153, 163, 554, 873, 1, 1143, 2027, 2028, 119, 874, 7], [1598, 1144, 1000, 20, 20, 349, 412], [555, 137, 25, 35, 2029, 329, 1318, 23, 35, 1318, 56, 208, 439, 439, 604, 98, 61, 1, 1319, 64, 5, 32

In [15]:
test_tokenizer = Tokenizer()
test_tokenizer.fit_on_texts(test_df['text'])
test_vocab = test_tokenizer.texts_to_sequences(test_df['text'])

In [16]:
X_train = pad_sequences(encoded_vocab, maxlen=sentence_length, padding='post')
X_test = pad_sequences(test_vocab, maxlen=sentence_length, padding='post')
Y_train = data_df['label']
Y_test = test_df['label']

In [17]:
wv = Word2Vec.load("./drive/MyDrive/aathesis/idwiki_word2vec_300.model")
embeddings = wv.wv

In [18]:
embedding_weight = np.vstack([
           np.zeros(embeddings.vectors.shape[1]),
           embeddings.vectors
])

In [ ]:
embedding_matrix = np.zeros((vocab_size, word2vec_len))
iv = 0
for word, idx in my_tokenizer.word_index.items():
  if word in embeddings:
    embedding_matrix[index, :] =  embeddings[word]
    iv += 1
print(f"tokens in vocab: {iv}")

tokens in vocab: 5193


In [19]:
models = list()
resample = list()
acc_list = list()
precision_list = list()
recall_list = list()
train_length = list()
test_length = list()

In [68]:
len(models), len(resample), len(acc_list), len(precision_list), len(recall_list), len(train_length), len(test_length)

(20, 20, 20, 20, 20, 20, 16)

In [47]:
def test_evals(clf_model, X_test, y_test, train_len, algo=None, sampling=None):
  y_pred=clf_model.predict(X_test)
  print('Confusion Matrix')
  print('='*60)
  print(confusion_matrix(y_test,y_pred),"\n")
  print('Classification Report')
  print('='*60)
  print(classification_report(y_test,y_pred),"\n")

  models.append(algo)
  acc_list.append(accuracy_score(y_test,y_pred))
  precision_list.append(precision_score(y_test,y_pred))
  recall_list.append(recall_score(y_test,y_pred))
  resample.append(sampling)
  train_length.append(train_len)
  test_length.append(X_test.shape[0])

In [24]:
def evals(clf_model, X_test, y_test, train_len, algo=None, sampling=None):
  # Test set prediction
  loss, accuracy,precision,recall = clf_model.evaluate(X_test,y_test)

  models.append(algo)
  acc_list.append(accuracy)
  precision_list.append(precision)
  recall_list.append(recall)
  resample.append(sampling)
  train_length.append(train_len)
  test_length.append(len(X_test))

In [ ]:
import os
from os import listdir
from os.path import isfile, join, isdir

In [ ]:
#building the cnn in keras
def build_cnns(sentence_length, word2vec_len):
  model = Sequential([
     Embedding(vocab_size, word2vec_len, weights=[embedding_matrix],
               input_length=sentence_length),
     Conv1D(128, 5, activation='relu'),
     GlobalMaxPooling1D(),
     Dense(20, activation='relu'),
     Dense(1, kernel_initializer='normal', activation='sigmoid')
     ])
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
  return model

In [ ]:
cnn_model = build_cnns(sentence_length, word2vec_len)
cnn_model.fit(X_train, Y_train, epochs = 10,batch_size = 32,validation_data = (X_test, Y_test),verbose = 1)

In [ ]:
loss, accuracy,precision,recall = cnn_model.evaluate(X_test,Y_test)
print(f"accuracy {accuracy},precision {precision},recall {recall}")

16/16 [==============================] - 0s 14ms/step - loss: 3.0500 - accuracy: 0.5500 - precision_1: 0.3433 - recall_1: 0.1127
accuracy 0.550000011920929,precision 0.34328359365463257,recall 0.11274509876966476


## Logistic Regression

In [73]:
logit = LogisticRegression(random_state=42, n_jobs=-1)
#train model
logit.fit(X_train,
          Y_train)
test_evals(logit, X_test, Y_test, X_train.shape[0], 'Log. Regression', 'No Aug')

Confusion Matrix
[[264  32]
 [ 80 124]] 

Classification Report
              precision    recall  f1-score   support

           0       0.77      0.89      0.82       296
           1       0.79      0.61      0.69       204

    accuracy                           0.78       500
   macro avg       0.78      0.75      0.76       500
weighted avg       0.78      0.78      0.77       500
 



In [74]:
logit.fit(sm_train_x, sm_train_y)
test_evals(logit, X_test, Y_test, sm_train_x.shape[0], 'Log. Regression', 'SMOTE')

Confusion Matrix
[[190 106]
 [ 18 186]] 

Classification Report
              precision    recall  f1-score   support

           0       0.91      0.64      0.75       296
           1       0.64      0.91      0.75       204

    accuracy                           0.75       500
   macro avg       0.78      0.78      0.75       500
weighted avg       0.80      0.75      0.75       500
 



In [75]:
logit.fit(ada_train_x, ada_train_y)
test_evals(logit, X_test, Y_test, ada_train_x.shape[0], 'Log. Regression', 'ADASYN')

Confusion Matrix
[[180 116]
 [ 13 191]] 

Classification Report
              precision    recall  f1-score   support

           0       0.93      0.61      0.74       296
           1       0.62      0.94      0.75       204

    accuracy                           0.74       500
   macro avg       0.78      0.77      0.74       500
weighted avg       0.81      0.74      0.74       500
 



In [ ]:
logit.fit(tom_train_x, tom_train_y)
test_evals(logit, X_test, Y_test, tom_train_x.shape[0], 'Log. Regression', 'SMOTE + Tomek')

In [ ]:
logit.fit(enn_train_x, enn_train_y)
test_evals(logit, X_test, Y_test, enn_train_x.shape[0], 'Log. Regression', 'SMOTE + ENN')

In [79]:
logit.fit(rus_train_x, rus_train_y)
test_evals(logit, X_test, Y_test, rus_train_x.shape[0], 'Log. Regression', 'Random Undersampling')

Confusion Matrix
[[115 181]
 [ 12 192]] 

Classification Report
              precision    recall  f1-score   support

           0       0.91      0.39      0.54       296
           1       0.51      0.94      0.67       204

    accuracy                           0.61       500
   macro avg       0.71      0.66      0.60       500
weighted avg       0.75      0.61      0.59       500
 



In [80]:
logit.fit(ros_train_x, ros_train_y)
test_evals(logit, X_test, Y_test, ros_train_x.shape[0], 'Log. Regression', 'Random Oversampling')

Confusion Matrix
[[179 117]
 [ 15 189]] 

Classification Report
              precision    recall  f1-score   support

           0       0.92      0.60      0.73       296
           1       0.62      0.93      0.74       204

    accuracy                           0.74       500
   macro avg       0.77      0.77      0.74       500
weighted avg       0.80      0.74      0.73       500
 



## XGBoost

In [ ]:
xgb_model = xgb.XGBClassifier(n_jobs=-1)

eval_set = [(X_test, Y_test)]
#train model
xgb_model.fit(X_train,
              Y_train,
              eval_set = [(X_test, Y_test)],
              verbose=True)

In [48]:
test_evals(xgb_model, X_test, Y_test, X_train.shape[0], 'XGBoost', 'No Aug')

Confusion Matrix
[[245  51]
 [ 52 152]] 

Classification Report
              precision    recall  f1-score   support

           0       0.82      0.83      0.83       296
           1       0.75      0.75      0.75       204

    accuracy                           0.79       500
   macro avg       0.79      0.79      0.79       500
weighted avg       0.79      0.79      0.79       500
 



In [ ]:
smote = SMOTE(random_state=42)
sm_train_x, sm_train_y = smote.fit_resample(X_train, Y_train)
xgb_model.fit(sm_train_x, sm_train_y,
              eval_set = [(X_test, Y_test)],
              verbose=True)

test_evals(xgb_model, X_test, Y_test, sm_train_x.shape[0], 'XGBoost', 'SMOTE')

In [ ]:
adasyn = ADASYN(random_state=42)
ada_train_x, ada_train_y = adasyn.fit_resample(X_train, Y_train)
xgb_model.fit(ada_train_x, ada_train_y,
              eval_set = [(X_test, Y_test)],
              verbose=True)
test_evals(xgb_model, X_test, Y_test, ada_train_x.shape[0], 'XGBoost', 'ADASYN')

In [ ]:
smtomek = SMOTETomek(random_state=42)
tom_train_x, tom_train_y = smtomek.fit_resample(X_train, Y_train)
xgb_model.fit(tom_train_x, tom_train_y,
              eval_set = [(X_test, Y_test)],
              verbose=True)
test_evals(xgb_model, X_test, Y_test, tom_train_x.shape[0], 'XGBoost', 'SMOTE + Tomek')

In [ ]:
sm_enn = SMOTEENN(random_state=42)
enn_train_x, enn_train_y = sm_enn.fit_resample(X_train, Y_train)
xgb_model.fit(enn_train_x, enn_train_y,
              eval_set = [(X_test, Y_test)],
              verbose=True)

test_evals(xgb_model, X_test, Y_test, enn_train_x.shape[0], 'XGBoost', 'SMOTE + ENN')

In [ ]:
rus = RandomUnderSampler(random_state=42)
rus_train_x, rus_train_y = rus.fit_resample(X_train, Y_train)
xgb_model.fit(rus_train_x, rus_train_y,
              eval_set = [(X_test, Y_test)],
              verbose=True)

test_evals(xgb_model, X_test, Y_test, rus_train_x.shape[0], 'XGBoost', 'Random Undersampling')

In [ ]:
ros = RandomOverSampler(random_state=42)
ros_train_x, ros_train_y = ros.fit_resample(X_train, Y_train)
xgb_model.fit(ros_train_x, ros_train_y,
              eval_set = [(X_test, Y_test)],
              verbose=True)

test_evals(xgb_model, X_test, Y_test, ros_train_x.shape[0], 'XGBoost', 'Random Oversampling')

## AdaBoost

In [58]:
adaboost_model = AdaBoostClassifier()

#train model
adaboost_model.fit(X_train,
                   Y_train)
test_evals(adaboost_model, X_test, Y_test, X_train.shape[0], 'AdaBoost', 'No Aug')

Confusion Matrix
[[252  44]
 [ 73 131]] 

Classification Report
              precision    recall  f1-score   support

           0       0.78      0.85      0.81       296
           1       0.75      0.64      0.69       204

    accuracy                           0.77       500
   macro avg       0.76      0.75      0.75       500
weighted avg       0.76      0.77      0.76       500
 



In [59]:
adaboost_model.fit(sm_train_x, sm_train_y)
test_evals(adaboost_model, X_test, Y_test, sm_train_x.shape[0], 'AdaBoost', 'SMOTE')

Confusion Matrix
[[110 186]
 [ 12 192]] 

Classification Report
              precision    recall  f1-score   support

           0       0.90      0.37      0.53       296
           1       0.51      0.94      0.66       204

    accuracy                           0.60       500
   macro avg       0.70      0.66      0.59       500
weighted avg       0.74      0.60      0.58       500
 



In [ ]:
adaboost_model.fit(ada_train_x, ada_train_y)
test_evals(adaboost_model, X_test, Y_test, ada_train_x.shape[0], 'AdaBoost', 'ADASYN')

In [ ]:
# smtomek = SMOTETomek(random_state=42)
# tom_train_x, tom_train_y = smtomek.fit_resample(X_train, Y_train)
adaboost_model.fit(tom_train_x, tom_train_y)
test_evals(adaboost_model, X_test, Y_test, tom_train_x.shape[0], 'AdaBoost', 'SMOTE + Tomek')

In [ ]:
# sm_enn = SMOTEENN(random_state=42)
# enn_train_x, enn_train_y = sm_enn.fit_resample(X_train, Y_train)
adaboost_model.fit(enn_train_x, enn_train_y)

test_evals(adaboost_model, X_test, Y_test, enn_train_x.shape[0], 'AdaBoost', 'SMOTE + ENN')

In [ ]:
# rus = RandomUnderSampler(random_state=42)
# rus_train_x, rus_train_y = rus.fit_resample(X_train, Y_train)
adaboost_model.fit(rus_train_x, rus_train_y)

test_evals(adaboost_model, X_test, Y_test, rus_train_y.shape[0], 'AdaBoost', 'Random Undersampling')

In [ ]:
adaboost_model.fit(ros_train_x, ros_train_y)

test_evals(adaboost_model, X_test, Y_test, ros_train_y.shape[0], 'AdaBoost', 'Random Oversampling')

## CNN

In [ ]:
#building the cnn in keras
def build_cnn(sentence_length, word2vec_len):
  model = Sequential([
     Embedding(vocab_size, word2vec_len, input_length=sentence_length),
     Conv1D(128, 5, activation='relu'),
     GlobalMaxPooling1D(),
     Dense(20, activation='relu'),
     Dense(1, kernel_initializer='normal', activation='sigmoid')
     ])
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
  return model

In [ ]:
params = {
}
scorers = {
        'precision_score': make_scorer(precision_score),
        'recall_score': make_scorer(recall_score),
        'accuracy_score': make_scorer(accuracy_score)
        }
cnn_model = build_cnn(sentence_length, word2vec_len)
# cnn_m = KerasClassifier(model = cnn_model, optimizer="adam", epochs=5, verbose=0)
# search = GridSearchCV(estimator=cnn_m, param_grid=params, cv=5, scoring = scorers,
#                       refit='accuracy_score', return_train_score=True)
cnn_model.fit(X_train, Y_train, epochs=5, verbose=True, shuffle=False)

Epoch 1/5
79/79 [==============================] - 13s 147ms/step - loss: 0.5576 - accuracy: 0.7175 - precision_6: 0.6375 - recall_6: 0.1360
Epoch 2/5
79/79 [==============================] - 2s 25ms/step - loss: 0.3077 - accuracy: 0.8808 - precision_6: 0.8096 - recall_6: 0.7880
Epoch 3/5
79/79 [==============================] - 2s 26ms/step - loss: 0.1173 - accuracy: 0.9532 - precision_6: 0.9451 - recall_6: 0.8960
Epoch 4/5
79/79 [==============================] - 2s 26ms/step - loss: 0.0353 - accuracy: 0.9872 - precision_6: 0.9825 - recall_6: 0.9747
Epoch 5/5
79/79 [==============================] - 2s 28ms/step - loss: 0.0170 - accuracy: 0.9924 - precision_6: 0.9854 - recall_6: 0.9893


In [ ]:
#predictions = cnn_model.predict(X_test)
#accuracy_score(Y_test, predictions)
evals(cnn_model, X_test, Y_test, len(X_test), 'CNN', 'No Aug')

16/16 [==============================] - 0s 6ms/step - loss: 2.6972 - accuracy: 0.5580 - precision_6: 0.3924 - recall_6: 0.1520


In [81]:
clf_eval_df = pd.DataFrame({'model':models,
                            'resample':resample,
                            'accuracy': acc_list,
                            'precision':precision_list,
                            'recall':recall_list,
                            'train length':train_length})

In [82]:
clf_eval_df

,model,resample,accuracy,precision,recall,train length
0,XGBoost,No Aug,0.794,0.748768,0.745098,2499
1,XGBoost,No Aug,0.794,0.748768,0.745098,2499
2,XGBoost,No Aug,0.794,0.748768,0.745098,2499
3,XGBoost,No Aug,0.794,0.748768,0.745098,2499
4,XGBoost,No Aug,0.794,0.748768,0.745098,2499
5,XGBoost,No Aug,0.794,0.748768,0.745098,2499
6,XGBoost,SMOTE,0.792,0.710084,0.828431,3498
7,XGBoost,ADASYN,0.796,0.725664,0.803922,3518
8,XGBoost,SMOTE + Tomek,0.792,0.696850,0.867647,3486
9,XGBoost,SMOTE + ENN,0.466,0.432836,0.995098,1795


In [85]:
clf_eval_df.to_csv("/content/drive/MyDrive/aathesis/clf_eval.csv", index = False)

## BiLSTM

In [ ]:
def get_x_y(df, num_classes, word2vec_len, input_size, word2vec):

  #read in lines
  train_lines = df
  num_lines = len(train_lines)

  #initialize x and y matrix
  x_matrix = None
  y_matrix = None

  try:
    x_matrix = np.zeros((num_lines, input_size, word2vec_len))
  except:
    print("Error!", num_lines, input_size, word2vec_len)
  y_matrix = np.zeros((num_lines, num_classes))

  #insert values
  for i, line in enumerate(train_lines):
    #parts = line[:-1].split('\t')
    label = int(train_lines['label'][i])
    sentence = train_lines['text'][i]

    #insert x
    words = sentence.split(' ')
    words = words[:x_matrix.shape[1]] #cut off if too long
    for j, word in enumerate(words):
      if word in word2vec:
        x_matrix[i, j, :] = word2vec[word]

    #insert y
    y_matrix[i][label] = 1.0
  return x_matrix, y_matrix

In [ ]:
#building the cnn in keras
def build_cnns(sentence_length, word2vec_len):
  model = Sequential([
     Embedding(vocab_size, word2vec_len, weights=[embedding_matrix],
               input_length=sentence_length),
     Conv1D(128, 5, activation='relu'),
     GlobalMaxPooling1D(),
     Dense(20, activation='relu'),
     Dense(1, kernel_initializer='normal', activation='sigmoid')
     ])
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
  return model

In [ ]:
def build_models(sentence_length, word2vec_len):
	model = Sequential([
     Embedding(vocab_size, word2vec_len, weights=[embedding_matrix],
               input_length=sentence_length, trainable=False),
     Bidirectional(LSTM(64, return_sequences=True)),
     Dropout(0.5),
     Bidirectional(LSTM(32, return_sequences=False)),
     Dropout(0.5),
     Dense(20, activation='relu'),
     Dense(1, activation='sigmoid')
     ])
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
	return model

In [ ]:
model_lstm = build_models(sentence_length, word2vec_len)

In [ ]:
#implement early stopping
callbacks = [EarlyStopping(monitor='val_loss', patience=10)]

#train model
model_lstm.fit(X_train, Y_train, epochs=10, callbacks=callbacks,
               validation_data = (X_test, Y_test),verbose = 1,
               batch_size=32,
               shuffle=False)

In [ ]:
loss, accuracy,precision,recall = model_lstm.evaluate(X_test,Y_test)
print(f"accuracy {accuracy},precision {precision},recall {recall}")

16/16 [==============================] - 1s 53ms/step - loss: 1.8533 - accuracy: 0.5700 - precision_4: 0.4068 - recall_4: 0.1176
accuracy 0.5699999928474426,precision 0.4067796468734741,recall 0.11764705926179886


In [ ]:
evals(model_lstm, test_x, test_y, len(train_x), 'Bi-LSTM', 'No Aug')

16/16 [==============================] - 0s 7ms/step - loss: 0.0320 - accuracy: 0.9960 - precision_5: 0.0000e+00 - recall_5: 0.0000e+00


In [ ]:
#evaluate model
y_preds = model_lstm.predict(test_x)
test_y_cat = one_hot_to_categorical(test_y)
y_pred_cat = one_hot_to_categorical(y_preds)
acc = accuracy_score(test_y_cat, y_pred_cat)

16/16 [==============================] - 2s 7ms/step


In [ ]:
acc

0.996

In [ ]:
smote_enn = SMOTEENN()
smote_enn_x, smote_enn_y = smote_enn.fit_resample(result, Y_train)

In [ ]:
smote_enn_x.shape, smote_enn_y.shape

((0, 30), (0,))

In [ ]:
### INPUT KE MODEL
train_x.shape, train_y.shape

((2499, 30, 300), (2499, 2))

In [ ]:
res_x_back = np.array(smote_enn_x).reshape(smote_enn_x.shape[0], 30, 300)

In [ ]:
y_matrix = np.zeros((smote_enn_x.shape[0], 2))
for i, line in enumerate(smote_enn_y):
  a = int(smote_enn_y[i])
  if a == 0:
    y_matrix[i][a] = 1
res_y_back = y_matrix

In [ ]:
smote_enn_x.shape

(1673, 30)

In [ ]:
#train model
model_lstm.fit(res_x_back, res_y_back, epochs=100, callbacks=callbacks,
               validation_split=0.1,
               batch_size=32,
               shuffle=False,
               verbose=1)

NameError: name 'model_lstm' is not defined

In [ ]:
evals(model_lstm, test_x, test_y, len(res_x_back), 'Bi-LSTM', 'SMOTE + ENN')

16/16 [==============================] - 0s 11ms/step - loss: 5.1219 - accuracy: 0.5920 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00


In [ ]:
word_vectors.key_to_index["syukur"]

7458

## GRU

In [ ]:
# Create BiLSTM model
def create_bilstm(sentence_length, word2vec_len):
    model = Sequential()
    # Input layer
    model.add(Bidirectional(
              LSTM(64, return_sequences=True),
              input_shape=(sentence_length, word2vec_len)))
    # Hidden layer
    model.add(Bidirectional(LSTM(64, return_sequences=False)))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    #Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
#building the gru in keras
def build_gru(sentence_length, word2vec_len):
  model = keras.Sequential()
  model.add(layers.GRU(64, input_shape=(sentence_length, word2vec_len)))
  model.add(layers.BatchNormalization())
  model.add(layers.Dense(20))
  model.add(layers.Dense(1, kernel_initializer='normal', activation='sigmoid'))

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
  return model

In [ ]:
gru_model = build_gru(sentence_length, word2vec_len)

In [ ]:
x_trains = X_train.reshape(-1, 2499, 30)
x_trains.shape

(1, 2499, 30)

In [ ]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in my_tokenizer.word_index.items():
	embedding_vector = word_vectors.key_to_index[word]
	if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [ ]:
my_tokenizer = TokenizerTransformer()
my_padder = PadSequencesTransformer(maxlen=30)
my_model = KerasClassifier(
               build_fn=create_model,
               epochs=100,
               embedding_input_dim=vocab_size,
               embedding_output_dim=EMBEDDING_DIM,
               embedding_weights=embedding_weights
)

pipeline = Pipeline([
              ('tokenizer', my_tokenizer),
              ('padder', my_padder),
              ('smote', SMOTE(random_state=42)),
              ('model', my_model)
])
pipeline.fit(data_df['text'], data_df['label'])

In [ ]:
my_tokenizer = TokenizerTransformer()
my_padder = PadSequencesTransformer(maxlen=longest_text)
my_model = KerasClassifier(
  model=create_model,
  epochs=200,
  embedding_input_dim=vocab_size,
  embedding_output_dim=EMBEDDING_DIM,
  embedding_weights=embedding_weights,
  batch_size=1024,
  validation_split=0.1,
  verbose=0)

pipelines = Pipeline([
  ('tokenizer', my_tokenizer),
  ('padder', my_padder),
  ('smote', SMOTE(random_state=42)),
  ('model', my_model)
])
pipelines.fit(data_df['text'], data_df['label'])


In [ ]:
#def evaluate():
test_df = pd.read_csv('/content/increment_datasets_f2/test.txt', sep='\t', header = None)
test_df.columns = ['label', 'text']
y_pred = pipeline.predict(test_df['text'])
acc = accuracy_score(test_df['label'], y_pred)

In [ ]:
acc

In [ ]:
no_smote = Pipeline([
  ('tokenizer', my_tokenizer),
  ('padder', my_padder),
  ('model', my_model)
])
no_smote.fit(data_df['text'], data_df['label'])

In [ ]:
ada_tokenizer = TokenizerTransformer()
ada_padder = PadSequencesTransformer(maxlen=30)
ada_model = KerasClassifier(
               build_fn=create_model,
               epochs=200,
               embedding_input_dim=vocab_size,
               embedding_output_dim=EMBEDDING_DIM,
               embedding_weights=embedding_weight
)

ada = Pipeline([
              ('tokenizer', ada_tokenizer),
              ('padder', ada_padder),
              ('smote', ADASYN()),
              ('model', ada_model)
])
ada.fit(data_df['text'], data_df['label'])

In [ ]:
ada_tokenizer = TokenizerTransformer()
ada_padder = PadSequencesTransformer(maxlen=30)
ada_model = KerasClassifier(
               build_fn=create_model,
               epochs=200,
               embedding_input_dim=vocab_size,
               embedding_output_dim=EMBEDDING_DIM,
               embedding_weights=embedding_weights
)

SMTomek = Pipeline([
              ('tokenizer', ada_tokenizer),
              ('padder', ada_padder),
              ('SMOTETomek', SMOTETomek()),
              ('model', ada_model)
])
SMTomek.fit(data_df['text'], data_df['label'])

In [ ]:
ada_tokenizer = TokenizerTransformer()
ada_padder = PadSequencesTransformer(maxlen=30)
ada_model = KerasClassifier(
               build_fn=create_model,
               epochs=200,
               embedding_input_dim=vocab_size,
               embedding_output_dim=EMBEDDING_DIM,
               embedding_weights=embedding_weights
)

SMENN = Pipeline([
              ('tokenizer', ada_tokenizer),
              ('padder', ada_padder),
              ('SMOTEENN', SMOTEENN()),
              ('model', ada_model)
])
SMENN.fit(data_df['text'], data_df['label'])

In [ ]:
# Make predictions on the test set
predictions = xgb.predict(test_df['text'])
accuracy_score(test_df['label'], predictions)

In [ ]:
y_pred = pipeline.predict(test_df['text'])
accuracy_score(test_df['label'], y_pred)

In [ ]:
y_preds = no_smote.predict(test_df['text'])
accuracy_score(test_df['label'], y_preds)

In [ ]:
y_preds = ada.predict(test_df['text'])
accuracy_score(test_df['label'], y_preds)

In [ ]:
y_pred_tomek = SMTomek.predict(test_df['text'])
accuracy_score(test_df['label'], y_pred_tomek)

In [ ]:
y_pred_smoteenn = SMENN.predict(test_df['text'])
accuracy_score(test_df['label'], y_pred_smoteenn)

In [ ]:
import joblib
joblib.dump(no_smote, '/content/drive/MyDrive/aathesis/save/no_smote.pkl')

In [ ]:
joblib.dump(ada, '/content/drive/MyDrive/aathesis/save/ada.pkl')
joblib.dump(SMTomek, '/content/drive/MyDrive/aathesis/save/SMTomek.pkl')
joblib.dump(SMENN, '/content/drive/MyDrive/aathesis/save/SMENN.pkl')

In [ ]:
pipeline = joblib.load(PIPE_PATH)

In [ ]:
def run_model(train_file, test_file, input_size, percent_dataset):

  #initialize model
  #model = build_cnn(input_size, word2vec_len, num_classes)

  model = xgb.XGBClassifier(n_jobs=-1)

  #load data
  data_df = text_to_df(train_file)
  test_df = text_to_df(test_file)

  my_tokenizer = TokenizerTransformer()
  my_padder = PadSequencesTransformer(maxlen=input_size)
  my_tokenizer.fit(data_df['text'])
  tokenized = my_tokenizer.transform(data_df['text'])
  my_padder.fit(tokenized)
  train_x = my_padder.transform(tokenized)
  train_y = data_df['label']

  test_tokenizer = TokenizerTransformer()
  test_padder = PadSequencesTransformer(maxlen=input_size)
  test_tokenizer.fit(test_df['text'])
  test_tokenized = test_tokenizer.transform(test_df['text'])
  test_padder.fit(test_tokenized)
  test_x = test_padder.transform(test_tokenized)
  test_y = test_df['label']

  #smote = SMOTE()
  #train_x, train_y = smote.fit_resample(X_train, train_y)
  eval_set = [(test_x, test_y)]
  #train model
  model.fit(train_x,
        train_y,
        eval_metric=["error", "logloss"],
        eval_set=eval_set,
        verbose=True)
  #model.save('checkpoints/lol')
  #model = load_model('checkpoints/lol')
  #evaluate model
  y_pred = model.predict(test_x)
  acc = accuracy_score(test_y, y_pred)
  results = model.evals_result()
  #return the accuracy
  #print("data with shape:", train_x.shape, train_y.shape, 'train=', train_file, 'test=', test_file, 'with fraction', percent_dataset, 'had acc', acc)
  return acc, results

In [ ]:
train_orig = '/content/drive/MyDrive/aathesis/increment_datasets_f2/pc/orig_full.txt'
test_path = '/content/drive/MyDrive/aathesis/increment_datasets_f2/pc/test.txt'

In [ ]:
accs, results= run_model(train_orig, test_path, 50, 1)

In [ ]:
my_tokenizer = TokenizerTransformer()
my_padder = PadSequencesTransformer(maxlen=30)
my_model = KerasClassifier(
  model=create_model,
  epochs=200,
  embedding_input_dim=vocab_size,
  embedding_output_dim=EMBEDDING_DIM,
  embedding_weights=embedding_weights,
  batch_size=1024,
  validation_split=0.1,
  verbose=True)


NameError: name 'create_model' is not defined

In [ ]:
models = build_cnn(30, 300, 2)

In [ ]:
models.fit(train_x, train_y, epochs = 100, verbose = True)

In [ ]:
models.predict(test_x)

In [ ]:
#get full image paths
def get_txt_paths(folder):
    txt_paths = [join(folder, f) for f in listdir(folder) if isfile(join(folder, f)) and '.txt' in f]
    if join(folder, '.DS_Store') in txt_paths:
        txt_paths.remove(join(folder, '.DS_Store'))
    txt_paths = sorted(txt_paths)
    return txt_paths

#get subfolders
def get_subfolder_paths(folder):
    subfolder_paths = [join(folder, f) for f in listdir(folder) if (isdir(join(folder, f)) and '.DS_Store' not in f)]
    if join(folder, '.DS_Store') in subfolder_paths:
        subfolder_paths.remove(join(folder, '.DS_Store'))
    subfolder_paths = sorted(subfolder_paths)
    return subfolder_paths

#get all image paths
def get_all_txt_paths(master_folder):

    all_paths = []
    subfolders = get_subfolder_paths(master_folder)
    if len(subfolders) > 1:
        for subfolder in subfolders:
            all_paths += get_txt_paths(subfolder)
    else:
        all_paths = get_txt_paths(master_folder)
    return all_paths

In [ ]:
wo2vec = gen_vocab_dicts('/content/drive/MyDrive/aathesis',
                        '/content/drive/MyDrive/aathesis/glove.6B.300d.txt')

In [ ]:
w2vec = gen_vocab_dicts('/content/drive/MyDrive/aathesis',
                        '/content/drive/MyDrive/aathesis/glove.6B.300d.txt')

['/content/drive/MyDrive/aathesis/increment_datasets_f2/test.txt']
3025 unique words found
1176 matches between unique words and word2vec dictionary


In [ ]:
def neural_net():
  # encoder = LabelEncoder()
  # encoder.fit(train_labels)
  # y_tra = encoder.transform(train_labels)
  # y_tes = encoder.transform(test_labels)
  # print(y_tra[1], y_tes[1])

  num_classes = np.max(Y_train) + 1
  y_tra = utils.to_categorical(Y_train, num_classes)
  y_tes = utils.to_categorical(Y_test, num_classes)
  print(y_tra.shape)

  batch_size = 32
  epochs = 10
  model = Sequential([
     Embedding(vocab_size, 300, input_length=30),
     Bidirectional(LSTM(64, return_sequences=True)),
     Dropout(0.5),
     Bidirectional(LSTM(32, return_sequences=False)),
     Dropout(0.5),
     Dense(20, activation='relu'),
     Dense(2, activation='sigmoid')
     ])

  model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

  history = model.fit(X_train, y_tra,
                      batch_size=batch_size,
                      epochs=epochs,
                      verbose=1,
                      validation_split=0.3)

  score = model.evaluate(X_test, y_tes,
                       batch_size=batch_size, verbose=1)
  return score[1], score[2], score[3]

In [ ]:
neural_net()

(2499, 2)
Epoch 1/10
55/55 [==============================] - 21s 219ms/step - loss: 0.5491 - accuracy: 0.7158 - precision_3: 0.7116 - recall_3: 0.7181 - val_loss: 0.4767 - val_accuracy: 0.7573 - val_precision_3: 0.7611 - val_recall_3: 0.7520
Epoch 2/10
55/55 [==============================] - 8s 151ms/step - loss: 0.3532 - accuracy: 0.8531 - precision_3: 0.8494 - recall_3: 0.8451 - val_loss: 0.3895 - val_accuracy: 0.8227 - val_precision_3: 0.8290 - val_recall_3: 0.8147
Epoch 3/10
55/55 [==============================] - 9s 166ms/step - loss: 0.1532 - accuracy: 0.9508 - precision_3: 0.9540 - recall_3: 0.9497 - val_loss: 0.5326 - val_accuracy: 0.8133 - val_precision_3: 0.8178 - val_recall_3: 0.8080
Epoch 4/10
55/55 [==============================] - 11s 189ms/step - loss: 0.0656 - accuracy: 0.9777 - precision_3: 0.9794 - recall_3: 0.9766 - val_loss: 0.5988 - val_accuracy: 0.8133 - val_precision_3: 0.8132 - val_recall_3: 0.8067
Epoch 5/10
55/55 [==============================] - 8s 144ms

(0.5419999957084656, 0.5419999957084656, 0.5419999957084656)

In [ ]:
import pickle

# save the iris classification model as a pickle file
model_pkl_file = "iris_classifier_model.pkl"

with open(model_pkl_file, 'wb') as file:
  pickle.dump(model, file)

In [ ]:
# load model from pickle file
with open(model_pkl_file, 'rb') as file:
  model = pickle.load(file)

In [ ]:
train_comments = data_df['text']
train_labels = data_df['label']
test_comments = test_df['text']
test_labels = test_df['label']

max_words = 30
tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_comments)

x_tra = tokenize.texts_to_matrix(train_comments, mode = 'tfidf')
x_tes = tokenize.texts_to_matrix(test_comments, mode = 'tfidf')

encoder = LabelEncoder()
encoder.fit(train_labels)
y_tra = encoder.transform(train_labels)
y_tes = encoder.transform(test_labels)

num_classes = np.max(y_tra) + 1
y_tra = utils.to_categorical(y_tra, num_classes)
y_tes = utils.to_categorical(y_tes, num_classes)

batch_size = 32
epochs = 40

In [ ]:
def neural_nets():
    model = Sequential([
     Embedding(vocab_size, 300, input_length=30),
     Bidirectional(LSTM(64, return_sequences=True)),
     Dropout(0.5),
     Bidirectional(LSTM(32, return_sequences=False)),
     Dropout(0.5),
     Dense(20, activation='relu'),
     Dense(2, activation='sigmoid')
     ])
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    history = model.fit(x_tra, y_tra,
                      batch_size=batch_size,
                      epochs=epochs,
                      verbose=1,
                      validation_split=0.3)
    score = model.evaluate(x_tes, y_tes,
                       batch_size=batch_size, verbose=1)
    return score[1], score[2], score[3]

In [ ]:
encoder = LabelEncoder()
encoder.fit(train_labels)
y_tra = encoder.transform(train_labels)
y_tes = encoder.transform(test_labels)
num_classes = np.max(y_tra) + 1
y_tra = utils.to_categorical(data_df['label'], num_classes)
y_tes = utils.to_categorical(test_df['label'], num_classes)

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

def create_model(vocab_size):
  model = Sequential([
     Embedding(vocab_size, 300, input_length=30),
     Bidirectional(LSTM(64, return_sequences=True)),
     Dropout(0.5),
     Bidirectional(LSTM(32, return_sequences=False)),
     Dropout(0.5),
     Dense(20, activation='relu'),
     Dense(2, activation='sigmoid')
     ])
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
  return model

In [ ]:
mmodel = create_model(vocab_size)
pipelines = Pipeline([
    ('vect',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('model', mmodel)
])

pipelines.fit(data_df['text'], data_df['label'])

ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueError: In

In [ ]:
neural_nets()

Epoch 1/40
55/55 [==============================] - 21s 216ms/step - loss: 0.6014 - accuracy: 0.6890 - precision_9: 0.6807 - recall_9: 0.7021 - val_loss: 0.5218 - val_accuracy: 0.7027 - val_precision_9: 0.7861 - val_recall_9: 0.5440
Epoch 2/40
55/55 [==============================] - 7s 130ms/step - loss: 0.5120 - accuracy: 0.7078 - precision_9: 0.6999 - recall_9: 0.7387 - val_loss: 0.5277 - val_accuracy: 0.6973 - val_precision_9: 0.6979 - val_recall_9: 0.6960
Epoch 3/40
55/55 [==============================] - 9s 158ms/step - loss: 0.5044 - accuracy: 0.7090 - precision_9: 0.7048 - recall_9: 0.7113 - val_loss: 0.5281 - val_accuracy: 0.7013 - val_precision_9: 0.7003 - val_recall_9: 0.6853
Epoch 4/40
55/55 [==============================] - 8s 146ms/step - loss: 0.5027 - accuracy: 0.6993 - precision_9: 0.7067 - recall_9: 0.6998 - val_loss: 0.5245 - val_accuracy: 0.7067 - val_precision_9: 0.7067 - val_recall_9: 0.7067
Epoch 5/40
55/55 [==============================] - 8s 136ms/step - los

(0.6039999723434448, 0.608433723449707, 0.6060000061988831)

In [ ]:
neural_nets()

Epoch 1/40
55/55 [==============================] - 18s 165ms/step - loss: 0.5868 - accuracy: 0.7084 - precision_6: 0.7060 - recall_6: 0.7084 - val_loss: 0.5512 - val_accuracy: 0.6893 - val_precision_6: 0.6765 - val_recall_6: 0.7027
Epoch 2/40
55/55 [==============================] - 8s 154ms/step - loss: 0.5034 - accuracy: 0.7187 - precision_6: 0.7163 - recall_6: 0.7233 - val_loss: 0.5203 - val_accuracy: 0.7067 - val_precision_6: 0.6817 - val_recall_6: 0.7680
Epoch 3/40
55/55 [==============================] - 7s 122ms/step - loss: 0.5096 - accuracy: 0.7084 - precision_6: 0.7035 - recall_6: 0.7176 - val_loss: 0.5286 - val_accuracy: 0.7067 - val_precision_6: 0.7052 - val_recall_6: 0.7080
Epoch 4/40
55/55 [==============================] - 9s 156ms/step - loss: 0.5018 - accuracy: 0.7176 - precision_6: 0.7169 - recall_6: 0.7284 - val_loss: 0.5244 - val_accuracy: 0.7093 - val_precision_6: 0.7030 - val_recall_6: 0.7227
Epoch 5/40
55/55 [==============================] - 7s 135ms/step - los

(0.5220000147819519, 0.5374045968055725, 0.7039999961853027)

In [25]:
X_train = data_df['text']
Y_train = data_df['label']
X_test = test_df['text']
Y_test = test_df['label']

In [26]:
pipeline = Pipeline([
    ('vect',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('xgb',XGBClassifier())
])
pipeline.fit(data_df['text'], data_df['label'])

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('xgb',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=None, n_jobs=None,
                               num_parallel_tree=None, random_state=None, ...))])

In [ ]:
y_preds = pipeline.predict(test_df['text'])

mean_f1 = accuracy_score(test_df['label'], y_preds)

In [30]:
test_evals(pipeline, test_df['text'], test_df['label'], len(data_df['text']), 'XGBoost', 'No Aug')

Confusion Matrix
[[245  51]
 [ 52 152]] 

Classification Report
              precision    recall  f1-score   support

           0       0.82      0.83      0.83       296
           1       0.75      0.75      0.75       204

    accuracy                           0.79       500
   macro avg       0.79      0.79      0.79       500
weighted avg       0.79      0.79      0.79       500
 

